### First step, we load the data

In [1]:
import numpy as np
import pandas as pd

import os

from pathlib import Path

input_root_dir = "data/food-101"
input_root_path = Path(input_root_dir)
print(os.listdir(input_root_dir))
image_dir_path = input_root_path/'images'

class_path = input_root_dir+'/meta/classes.txt'
train_img_name_path = input_root_dir+'/meta/train.txt'
test_img_name_path = input_root_dir+'/meta/test.txt'

['.DS_Store', 'images', 'meta', 'license_agreement.txt', 'README.txt']


In [2]:
def file2list(path):
    file1 = open(path,'r')
    lines = file1.readlines()
    final_list = [line.strip() for line in lines]
    return final_list

In [3]:
from sklearn import preprocessing

classes = file2list(class_path)
train_data = file2list(train_img_name_path)
test_data = file2list(test_img_name_path)
le = preprocessing.LabelEncoder()
targets = le.fit_transform(classes)

In [4]:
from torch.utils.data import Dataset

class FoodData(Dataset):
    def __init__(self,img_path,img_dir,size,transform=None):
        self.img_path = img_path
        self.img_dir = img_dir
        self.transform = transform
        self.size = size
#         self.mode = mode
        
    def __len__(self):
        return len(self.img_path)
    
    def __getitem__(self,index):
        label,img_name = self.img_path[index].split('/')
        path = self.img_dir+'/images/'+label+'/'+img_name+'.jpg'
        img = cv2.imread(path)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img = cv2.resize(img,(self.size,self.size))
        if self.transform:
            img = self.transform(img)
        return {
                'gt': img,
                'label': torch.tensor(le.transform([label])[0])
            }

/Users/olesiaelfimova/miniforge3/envs/cs762/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# from PIL import Image, ImageEnhance, ImageOps
import numpy as np
import random


# class ImageNetPolicy(object):
#     """ Randomly choose one of the best 24 Sub-policies on ImageNet.
#         Example:
#         >>> policy = ImageNetPolicy()
#         >>> transformed = policy(image)
#         Example as a PyTorch Transform:
#         >>> transform=transforms.Compose([
#         >>>     transforms.Resize(256),
#         >>>     ImageNetPolicy(),
#         >>>     transforms.ToTensor()])
#     """
#     def __init__(self, fillcolor=(128, 128, 128)):
#         self.policies = [
#             SubPolicy(0.4, "posterize", 8, 0.6, "rotate", 9, fillcolor),
#             SubPolicy(0.6, "solarize", 5, 0.6, "autocontrast", 5, fillcolor),
#             SubPolicy(0.8, "equalize", 8, 0.6, "equalize", 3, fillcolor),
#             SubPolicy(0.6, "posterize", 7, 0.6, "posterize", 6, fillcolor),
#             SubPolicy(0.4, "equalize", 7, 0.2, "solarize", 4, fillcolor),

#             SubPolicy(0.4, "equalize", 4, 0.8, "rotate", 8, fillcolor),
#             SubPolicy(0.6, "solarize", 3, 0.6, "equalize", 7, fillcolor),
#             SubPolicy(0.8, "posterize", 5, 1.0, "equalize", 2, fillcolor),
#             SubPolicy(0.2, "rotate", 3, 0.6, "solarize", 8, fillcolor),
#             SubPolicy(0.6, "equalize", 8, 0.4, "posterize", 6, fillcolor),

#             SubPolicy(0.8, "rotate", 8, 0.4, "color", 0, fillcolor),
#             SubPolicy(0.4, "rotate", 9, 0.6, "equalize", 2, fillcolor),
#             SubPolicy(0.0, "equalize", 7, 0.8, "equalize", 8, fillcolor),
#             SubPolicy(0.6, "invert", 4, 1.0, "equalize", 8, fillcolor),
#             SubPolicy(0.6, "color", 4, 1.0, "contrast", 8, fillcolor),

#             SubPolicy(0.8, "rotate", 8, 1.0, "color", 2, fillcolor),
#             SubPolicy(0.8, "color", 8, 0.8, "solarize", 7, fillcolor),
#             SubPolicy(0.4, "sharpness", 7, 0.6, "invert", 8, fillcolor),
#             SubPolicy(0.6, "shearX", 5, 1.0, "equalize", 9, fillcolor),
#             SubPolicy(0.4, "color", 0, 0.6, "equalize", 3, fillcolor),

#             SubPolicy(0.4, "equalize", 7, 0.2, "solarize", 4, fillcolor),
#             SubPolicy(0.6, "solarize", 5, 0.6, "autocontrast", 5, fillcolor),
#             SubPolicy(0.6, "invert", 4, 1.0, "equalize", 8, fillcolor),
#             SubPolicy(0.6, "color", 4, 1.0, "contrast", 8, fillcolor),
#             SubPolicy(0.8, "equalize", 8, 0.6, "equalize", 3, fillcolor)
#         ]


#     def __call__(self, img):
#         policy_idx = random.randint(0, len(self.policies) - 1)
#         return self.policies[policy_idx](img)

#     def __repr__(self):
#         return "AutoAugment ImageNet Policy"

In [6]:
# class SubPolicy(object):
#     def __init__(self, p1, operation1, magnitude_idx1, p2, operation2, magnitude_idx2, fillcolor=(128, 128, 128)):
#         ranges = {
#             "shearX": np.linspace(0, 0.3, 10),
#             "shearY": np.linspace(0, 0.3, 10),
#             "translateX": np.linspace(0, 150 / 331, 10),
#             "translateY": np.linspace(0, 150 / 331, 10),
#             "rotate": np.linspace(0, 30, 10),
#             "color": np.linspace(0.0, 0.9, 10),
#             "posterize": np.round(np.linspace(8, 4, 10), 0).astype(np.int),
#             "solarize": np.linspace(256, 0, 10),
#             "contrast": np.linspace(0.0, 0.9, 10),
#             "sharpness": np.linspace(0.0, 0.9, 10),
#             "brightness": np.linspace(0.0, 0.9, 10),
#             "autocontrast": [0] * 10,
#             "equalize": [0] * 10,
#             "invert": [0] * 10
#         }

#         # from https://stackoverflow.com/questions/5252170/specify-image-filling-color-when-rotating-in-python-with-pil-and-setting-expand
#         def rotate_with_fill(img, magnitude):
#             rot = img.convert("RGBA").rotate(magnitude)
#             return Image.composite(rot, Image.new("RGBA", rot.size, (128,) * 4), rot).convert(img.mode)

#         func = {
#             "shearX": lambda img, magnitude: img.transform(
#                 img.size, Image.AFFINE, (1, magnitude * random.choice([-1, 1]), 0, 0, 1, 0),
#                 Image.BICUBIC, fillcolor=fillcolor),
#             "shearY": lambda img, magnitude: img.transform(
#                 img.size, Image.AFFINE, (1, 0, 0, magnitude * random.choice([-1, 1]), 1, 0),
#                 Image.BICUBIC, fillcolor=fillcolor),
#             "translateX": lambda img, magnitude: img.transform(
#                 img.size, Image.AFFINE, (1, 0, magnitude * img.size[0] * random.choice([-1, 1]), 0, 1, 0),
#                 fillcolor=fillcolor),
#             "translateY": lambda img, magnitude: img.transform(
#                 img.size, Image.AFFINE, (1, 0, 0, 0, 1, magnitude * img.size[1] * random.choice([-1, 1])),
#                 fillcolor=fillcolor),
#             "rotate": lambda img, magnitude: rotate_with_fill(img, magnitude),
#             "color": lambda img, magnitude: ImageEnhance.Color(img).enhance(1 + magnitude * random.choice([-1, 1])),
#             "posterize": lambda img, magnitude: ImageOps.posterize(img, magnitude),
#             "solarize": lambda img, magnitude: ImageOps.solarize(img, magnitude),
#             "contrast": lambda img, magnitude: ImageEnhance.Contrast(img).enhance(
#                 1 + magnitude * random.choice([-1, 1])),
#             "sharpness": lambda img, magnitude: ImageEnhance.Sharpness(img).enhance(
#                 1 + magnitude * random.choice([-1, 1])),
#             "brightness": lambda img, magnitude: ImageEnhance.Brightness(img).enhance(
#                 1 + magnitude * random.choice([-1, 1])),
#             "autocontrast": lambda img, magnitude: ImageOps.autocontrast(img),
#             "equalize": lambda img, magnitude: ImageOps.equalize(img),
#             "invert": lambda img, magnitude: ImageOps.invert(img)
#         }

#         self.p1 = p1
#         self.operation1 = func[operation1]
#         self.magnitude1 = ranges[operation1][magnitude_idx1]
#         self.p2 = p2
#         self.operation2 = func[operation2]
#         self.magnitude2 = ranges[operation2][magnitude_idx2]


#     def __call__(self, img):
#         if random.random() < self.p1: img = self.operation1(img, self.magnitude1)
#         if random.random() < self.p2: img = self.operation2(img, self.magnitude2)
#         return img

In [7]:
# from autoaugment import ImageNetPolicy
# from albumentations.pytorch import ToTensorV2
# import albumentations as A
# augmentation_pipeline = A.Compose(
#     [
#         A.HorizontalFlip(p = 0.5), # apply horizontal flip to 50% of images
#         A.VerticalFlip(p =0.5),
#         A.OneOf(
#             [
#                 # apply one of transforms to 50% of images
#                 A.RandomContrast(), # apply random contrast
#                 A.RandomGamma(), # apply random gamma
#                 A.RandomBrightness(), # apply random brightness
#                 A.RandomBrightnessContrast(),
#             ],
#             p = 0.5
#         ),
#         A.Cutout(num_holes=10, max_h_size=20, max_w_size=20, fill_value=0, p=0.5),
#         A.Blur(blur_limit=(15, 15), p=0.5),
#         A.Normalize(
#             mean=[0.485, 0.456, 0.406],
#             std=[0.229, 0.224, 0.225]),
        
#         ToTensorV2() # convert the image to PyTorch tensor
#     ],
#     p = 1
# )
# transforms_train = transforms.Compose([
#     transforms.ToPILImage(),
#     transforms.RandomRotation(90),
# #     transforms.CenterCrop(10),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomVerticalFlip(),
#     transforms.ColorJitter(),
# #     ImageNetPolicy(),
    
#     transforms.ToTensor(),
#     transforms.Normalize( mean = np.array([0.485, 0.456, 0.406]),
#     std = np.array([0.229, 0.224, 0.225]))
# ])
# train_transforms = transforms.Compose([transforms.ToPILImage(),
#                                         transforms.RandomRotation(30),
#                                        transforms.RandomResizedCrop(224),
#                                        transforms.RandomHorizontalFlip(),ImageNetPolicy(),
#                                        transforms.ToTensor(),
#                                        transforms.Normalize([0.485, 0.456, 0.406],
#                                                             [0.229, 0.224, 0.225])])
# train_dataset = FoodData(train_data,input_root_dir,256,transforms_train)
train_dataset = FoodData(train_data,input_root_dir,256,None)

In [8]:
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader

batch = 64
valid_size = 0.2
num = train_data.__len__()
# Dividing the indices for train and cross validation
indices = list(range(num))
np.random.shuffle(indices)
split = int(np.floor(valid_size*num))
train_idx,valid_idx = indices[split:], indices[:split]

#Create Samplers
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = DataLoader(train_dataset, batch_size = batch, sampler = train_sampler)
valid_loader = DataLoader(train_dataset, batch_size = batch, sampler = valid_sampler)

In [10]:
# test_transforms = transforms.Compose([transforms.ToPILImage(),
#                                       transforms.CenterCrop(224),
#                                       transforms.ToTensor(),
#                                       transforms.Normalize([0.485, 0.456, 0.406],
#                                                            [0.229, 0.224, 0.225])])

# test_data = FoodData(test_data,input_root_dir,256,transform = test_transforms)
test_data = FoodData(test_data,input_root_dir,256,transform=None)

test_loader = DataLoader(test_data, batch_size=batch, shuffle=False)

### Define and train a big, teacher neural net

First we import the PyTorch libraries, then create the neural network.

However, instead of training now, you have the option of just loading the network I previously trained

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from models import *
from plot_funcs import *

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

In [ ]:
class linear_net(nn.Module):
    def __init__(self, dropout=0.5):
        super(linear_net, self).__init__()
        self.linear_1 = nn.Linear(784, 1200)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout)
        self.linear_2 = nn.Linear(1200, 1200)
        self.dropout = nn.Dropout(p=dropout)
        self.linear_3 = nn.Linear(1200, 10)

    def forward(self, input):
        scores = self.linear_1(input)
        scores = self.relu(scores)
        scores = self.linear_2(scores)
        scores = self.relu(scores)
        scores = self.dropout(scores)
        scores = self.linear_3(scores)
        return scores

In [ ]:
# Create our teacher model
big_model = linear_net().to(device)

In [ ]:
# Load our pre-trained teacher model
load_path = "teacher_linear_model/"
checkpoint = torch.load(load_path + "modelo", map_location=torch.device('cpu'))
big_model.load_state_dict(checkpoint['model_state_dict'])
big_model.eval()

In [ ]:
# This cell is here if you want to train the teacher model yourself
# if you don't want to lose time DO NOT RUN

from tqdm import tqdm

# Loss function
loss_fn = nn.CrossEntropyLoss()
# Create optimizer
lr = 5e-3
optimizer = Adam(big_model.parameters(), lr=lr)
epochs = 2
train_loss = []
train_acc = []
val_acc = []
it = 0
title = "progress"
for epoch in range(epochs):
    it_per_epoch = 0
    for features, labels in tqdm(train_loader):
        scores = big_model(features)
        loss = loss_fn(scores, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
        if it % 100 == 0 and it != 0 and len(train_acc) > 1:
            train_acc.append(evaluate(big_model, train_loader, max_ex=100))
            val_acc.append(evaluate(big_model, val_loader))
            plot_loss(train_loss, it, it_per_epoch, base_name="loss_"+title, title=title)
            plot_acc(train_acc, val_acc, it, it_per_epoch, base_name="acc_"+title, title=title)
        it += 1
        it_per_epoch += 1
#perform last book keeping
train_acc.append(evaluate(big_model, train_loader, max_ex=100))
val_acc.append(evaluate(big_model, val_loader))

In [ ]:
train_acc = evaluate(big_model, train_loader)
print("\nTrain accuracy: %.2f%%" % train_acc)
val_acc = evaluate(big_model,val_loader)
print("Validation accuracy: %.2f%%" % val_acc)
test_acc = evaluate(big_model, test_loader)
print("Test accuracy: %.2f%%" % test_acc)

### Define and train a small, student neural net

In [ ]:
class small_linear_net(nn.Module):
    def __init__(self):
        super(small_linear_net, self).__init__()
        self.linear_1 = nn.Linear(784, 50)
        self.relu = nn.ReLU()
        self.linear_2 = nn.Linear(50, 10)

    def forward(self, input):
        scores = self.linear_1(input)
        scores = self.relu(scores)
        scores = self.linear_2(scores)
        return scores

In [ ]:
# Create our student model
small_model = small_linear_net().to(device)

In [ ]:
# Load our pre-trained student model
# This is just if you want to check the accuracy of this model
# trained with the original MNIST data
load_path = "small_linear_model/"
checkpoint = torch.load(load_path + "modelo", map_location=torch.device('cpu'))
small_model.load_state_dict(checkpoint['model_state_dict'])
small_model.eval()

In [ ]:
train_acc = evaluate(small_model, train_loader)
print("\nTrain accuracy: %.2f%%" % train_acc)
val_acc = evaluate(small_model,val_loader)
print("Validation accuracy: %.2f%%" % val_acc)
test_acc = evaluate(small_model, test_loader)
print("Test accuracy: %.2f%%" % test_acc)

### Distillation training

In [ ]:
# Set output directory and create if needed
import os
output_dir = "small_linear_model_distill1/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
### Define our custom loss function
softmax_op = nn.Softmax(dim=1)
mseloss_fn = nn.MSELoss()

def my_loss(scores, targets, T=5):
    soft_pred = softmax_op(scores / T)
    soft_targets = softmax_op(targets / T)
    loss = mseloss_fn(soft_pred, soft_targets)
    return loss

In [ ]:
# Create a new student model to start training from zero
small_model = small_linear_net().to(device)
from tqdm import tqdm
%matplotlib inline

# Hyperparameters
lr = 5e-3
epochs = 5
temp = 5

# Create optimizer
optimizer = Adam(small_model.parameters(), lr=lr)
val_acc = []
train_acc = []
train_loss = [0]  # loss at iteration 0
for epoch in range(epochs):
    for features, labels in tqdm(train_loader):
        scores = small_model(features)
        targets = big_model(features)
        loss = my_loss(scores, targets, T = temp)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Book-keeping
        if it % 100 == 0:
            train_acc.append(evaluate(small_model, train_loader, max_ex=100))
            val_acc.append(evaluate(small_model, val_loader))
        it += 1
#perform last book-keeping
train_acc.append(evaluate(small_model, train_loader, max_ex=100))
val_acc.append(evaluate(small_model, val_loader))
plot_acc(train_acc, val_acc, it, it_per_epoch)

In [ ]:
train_acc = evaluate(small_model, train_loader)
print("\nTrain accuracy: %.2f%%" % train_acc)
val_acc = evaluate(small_model,val_loader)
print("Validation accuracy: %.2f%%" % val_acc)
test_acc = evaluate(small_model, test_loader)
print("Test accuracy: %.2f%%" % test_acc)